In [1]:
import numpy  as  np
import subprocess
import math
from pyDOE import *
from smt.sampling_methods import LHS
from configparser import ConfigParser

## Ground-truth data
strain_fe_media = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35]
stress_fe_media = [0, 0.00169707, 0.00336693, 0.00501758, 0.00665913, 0.00830489, 0.00997227, 0.0116837, 0.0134675, 0.0153589, 0.0174013, 0.0196476, 0.0221622, 0.0250236, 0.028327, 0.0321884, 0.0367485, 0.0421783, 0.0486846, 0.0565163, 0.0659717, 0.0774054, 0.0912357, 0.107951, 0.128119, 0.152385, 0.181485, 0.216236, 0.257543, 0.306386, 0.363809, 0.430904, 0.508782, 0.598541, 0.701216, 0.817721]
  
stress_fe_media  = np.array(stress_fe_media)[list(np.arange(0, len(stress_fe_media), 2))]
strain_fe_media  = np.array(strain_fe_media)[list(np.arange(0, len(strain_fe_media), 2))]

initial_coeffs = [0.3,0,0,0,0,1050]

def max_coeff(order):
    '''
    Approximating coefficients of polynomial based on the order 
    '''
    return (initial_coeffs[-1]*strain_fe_media[-1]**6)/(strain_fe_media[-1]**order)


c1_max = max_coeff(0)
c2_max = max_coeff(1)
c3_max = max_coeff(2)
c4_max = max_coeff(3)
c5_max = max_coeff(4)
c6_max = initial_coeffs[5]*1.5


## Sample coefficients by Latin Hypercube Sampling between 0 and their max values 
c1_range = LHS(xlimits = np.array([[0, 2], [0, 2]]))(5)[:, 0]
c2_range = LHS(xlimits = np.array([[0, 3], [0, 3]]))(5)[:, 0]
c3_range = LHS(xlimits = np.array([[0, 3], [0, 3]]))(5)[:, 0]
c4_range = LHS(xlimits = np.array([[0, 3], [0, 3]]))(5)[:, 0]
c5_range = LHS(xlimits = np.array([[0, 3], [0, 3]]))(5)[:, 0]
c6_range = LHS(xlimits = np.array([[800.0, 1200.0], [800.0, 1200.0]]))(400)[:, 0]

In [2]:
def parameter_space_equal(N):
    '''
    Create parameter space by sampling equal N samples by LHS for each coefficient
    '''
    C = np.meshgrid(LHS(c1_max,N), LHS(c2_max,N), LHS(c3_max,N), LHS(c4_max,N), LHS(c5_max,N), LHS(c6_max,N))
    return np.vstack((C[0].flatten(), C[1].flatten(),C[2].flatten(),C[3].flatten(),C[4].flatten(),C[5].flatten())).T


def parameter_space_different(c1_range,c2_range,c3_range,c4_range,c5_range,c6_range):
    '''
    Create parameter space by sampling different number of samples by LHS for each coefficient 
    '''
    coefficient_space = []
    for c1 in c1_range:
        for c2 in c2_range:
            for c3 in c3_range:
                for c4 in c4_range:
                    for c5 in c5_range:
                        for c6 in c6_range:
                            coefficient_space.append([c1,c2,c3,c4,c5,c6])

    return np.array(coefficient_space)


def sampling_from_ps(par_space, N_samples):
    '''
    Sample from parameter space by LH sampling of indecies
    '''
    indecies = (lhs(N_samples, samples = 1) * par_space.shape[0]).squeeze()
    indecies = [int(i) for i in indecies]
    return par_space[indecies]


coefficient_space_different = parameter_space_different(c1_range,c2_range,c3_range,c4_range,c5_range,c6_range)
sampled_coefficients = sampling_from_ps(coefficient_space_different, 100)
np.savetxt("sampled_coefficients.txt",sampled_coefficients)

In [44]:
def update_config_file(coeffs):
    '''
    Update coefficients in the config file of ISR3D for strain test
    '''
    str_coeffs = [str(c) for c in coeffs ] 
    config.set('main', 'strain_force_c1', str_coeffs[0])
    config.set('main', 'strain_force_c2', str_coeffs[1])
    config.set('main', 'strain_force_c3', str_coeffs[2])
    config.set('main', 'strain_force_c4', str_coeffs[3])
    config.set('main', 'strain_force_c5', str_coeffs[4])
    config.set('main', 'strain_force_c6', str_coeffs[5])

    with open('./cxa/stage1.strain_test.cfg', 'w') as f:
        config.write(f)


config = ConfigParser(allow_no_value=True, delimiters=(" "))
config.read('./cxa/stage1.strain_test.cfg')


for coeffs in sampled_coefficients:
    update_config_file(coeffs)
    subprocess.run(["./kernel/absmc/build/uniaxialStrainTest", "./cxa/stage1.strain_test.cfg"], stdout = subprocess.PIPE,universal_newlines = True).stdout
